# Project Demonstration
## Dashboard for Seismic Risk Index Map Based on Building Vulnerability

Notes: 
- This python notebook is only for demonstration on how the program runs. Complete instruction on how to use this program can be seen in [README.md](./README.md) file.
- This demonstration is only for `job_xml_generator` and `oq_calculation` programs as the `streamlit_app` program cannot be run in python notebook.

## job_xml_generator.py

- Prepare all parameters/setting that will be inputted in this program. As a sample, [samples_parameter](samples/samples_parameter.xlsx) is prepared in [samples](./samples/) folder as an example to guide you running this program.
- Create [data](./data/) folder.
- Prepare `csv` file that has information about exposure model that will be defined in `exposure_model.xml`. This file should be stored in [data](./data/) folder. Information about how to create the `csv` file could be accessed in [OpenQuake_docs](https://docs.openquake.org/oq-engine/manual/latest/risk.html). As a sample, a [csv_file](data/new_exposure_model.csv) is also prepared in [samples/data](./samples/data/) folder.
- To run this program, click `Run` button or push `shift+enter` on keyboard for each cell below.

In [ ]:
"""
The program job_xml_generator.py is a program designed to 
generate NRML data model (XML-based data) and configuration file.

Brief description of the main functions in this module are,
- xml_exposure_model()
    Call exposure_model_user_input() and define it as a variable user_input,
    return create_exposure_model_xml(*user_input)
- exposure_model_user_input(): 
    Returns all user inputs as parameters in generating exposure_model.xml
- create_exposure_model_xml(*user_inputs):
    Generate exposure_model.xml and save it to a folder.
- xml_fragility_model()
    Call fragility_model_input() and define it as a variable user_input,
    return create_fragility_model_xml(*user_input)
- fragility_model_input(): 
    Generate fragility_model.csv and save it to a folder and 
    returns all user inputs as parameters in generating fragility_model.xml
- create_fragility_model_xml(*user_inputs):
    Generate fragility_model.xml and save it to a folder.
- xml_rupture_model()
    Call rupture_model_input() and define it as a variable user_input,
    return create_rupture_model_xml(*user_input)
- rupture_model_input(): 
    Returns all user inputs as parameters in generating rupture_model.xml
- create_rupture_model_xml(*user_inputs):
    Generate rupture_model.xml and save it to a folder.
- job_ini()
    Call job_ini_input() and define it as a variable user_input,
    return create_job_ini(*user_input)
- job_ini_input(): 
    Returns all user inputs as parameters in generating job.ini
- create_job_ini(*user_inputs):
    Generate job.ini and save it to a folder.
"""

In [1]:
import os
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import user_input as ipt

In [2]:
# Declare folder name
data_folder = "data"
output_folder = "output"

In [3]:
# Create XML file for exposure model
def create_exposure_model_xml(
    unit_cost,
    unit_area,
    exposure_id,
    exposure_description,
    structural_type,
    is_retrofitted,
    limit_types,
    deductible_types,
    non_structural_type,
    business_type,
    contents_type,
    area_type,
    occupancy_periods,
    tag_names,
    assets_name):

    # Create the root element
    root_exposure_model = ET.Element("nrml")
    root_exposure_model.attrib = {}
    root_exposure_model.attrib["xmlns:gml"] = "http://www.opengis.net/gml"
    root_exposure_model.attrib["xmlns"] = "http://openquake.org/xmlns/nrml/0.4"
    
    # Create sub-elements and add to the root
    exp_model = ET.SubElement(root_exposure_model, "exposureModel")
    exp_model.attrib = {}
    exp_model.attrib["id"] = exposure_id
    exp_model.attrib["category"] = "buildings"
    exp_model.attrib["taxonomySource"] = "GEM taxonomy"

    exp_description = ET.SubElement(exp_model, "description")
    exp_description.text = exposure_description
    
    conversions = ET.SubElement(exp_model, "conversions")
    cost_types = ET.SubElement(conversions, "costTypes")
    
    # Structural cost type
    if structural_type is not None:
        structural_cost_types = ET.SubElement(cost_types, "costType")

        structural_cost_types.attrib = {}
        structural_cost_types.attrib["name"] = "structural"
        structural_cost_types.attrib["type"] = structural_type
        structural_cost_types.attrib["unit"] = unit_cost
        
        #if is_retrofitted is not None:
         #   structural_cost_types.attrib["retrofittedType"] = is_retrofitted
          #  structural_cost_types.attrib["retrofittedUnit"] = unit_cost
        
        types_to_boolean_map = {"absolute": "true", "relative": "false"}

        if limit_types is not None:
            limit = ET.SubElement(conversions, "insuranceLimit")
            limit.attrib = {}
            limit.attrib["isAbsolute"] = types_to_boolean_map[limit_types]
        if deductible_types is not None:
            limit = ET.SubElement(conversions, "deductible")
            limit.attrib = {}
            limit.attrib["isAbsolute"] = types_to_boolean_map[deductible_types]

    # Non-structural cost type
    if non_structural_type is not None:
        nonstructural_cost_types = ET.SubElement(cost_types, "costType")
        nonstructural_cost_types.attrib = {}
        nonstructural_cost_types.attrib["name"] = "non_structural"
        nonstructural_cost_types.attrib["type"] = non_structural_type
        nonstructural_cost_types.attrib["unit"] = unit_cost
        
    # Contents cost type
    if contents_type is not None:
        contents_cost_types = ET.SubElement(cost_types, "costType")
        contents_cost_types.attrib = {}
        contents_cost_types.attrib["name"] = "contents"
        contents_cost_types.attrib["type"] = contents_type
        contents_cost_types.attrib["unit"] = unit_cost

    # Business cost type
    if business_type is not None:
        business_cost_types = ET.SubElement(cost_types, "costType")
        business_cost_types.attrib = {}
        business_cost_types.attrib["name"] = "business"
        business_cost_types.attrib["type"] = business_type
        business_cost_types.attrib["unit"] = unit_cost

    # Area type
    if area_type is not None:
        area = ET.SubElement(conversions, "Area")
        area.attrib = {}
        area.attrib["type"] = area_type
        area.attrib["unit"] = unit_area

    # Occupancy periods
    occupancy_periods_se = ET.SubElement(exp_model, "occupancyPeriods")
    if occupancy_periods is not None:
        occupancy_periods_se.text = occupancy_periods
    
    # Tag names
    tag_names_se = ET.SubElement(exp_model, "tagNames")
    if tag_names is not None:
        tag_names_se.text = tag_names

    # Assets
    assets = ET.SubElement(exp_model, "assets")
    assets.text = assets_name
    
    print("\n========================================================")
    print("\nHere is your XML file!")
    
    # Convert the XML to a pretty-printed string
    exposure_model = ET.tostring(root_exposure_model, encoding="utf8", method="xml")
    print(BeautifulSoup(exposure_model, "xml").prettify())
    
    # Create the XML file
    tree_exposure_model = ET.ElementTree(root_exposure_model)
    file = os.path.join(data_folder, "exposure_model" + ".xml")
    tree_exposure_model.write(file)
     
    print("\n================= Process Completed ====================")
    print("Saved " + file)
    
    return end_of_process()

# Create XML file for fragility model
def create_fragility_model_xml(frag_model_id, loss_cat, description, num_of_frag_func, frag_func_id,
                               no_damage_lim, min_IML, max_IML, slight_mean, slight_stddev,
                               moderate_mean, moderate_stddev, extensive_mean, extensive_stddev,
                               complete_mean, complete_stddev):
    
    """ 
    Return XML file for building fragility model
    
    """
    
    # Create the root element
    root_frag_model = ET.Element('nrml')
    root_frag_model.attrib = {"xmlns":"http://openquake.org/xmlns/nrml/0.5"}

    # Create sub-elements and add to the root
    frag_model = ET.SubElement(root_frag_model, 'fragilityModel')
    frag_model.attrib = {}
    frag_model.attrib["id"] = frag_model_id
    frag_model.attrib["assetCategory"] = "buildings"
    frag_model.attrib["lossCategory"] = loss_cat
    
    desciption = ET.SubElement(frag_model, "description")
    desciption.text = description
    
    limit_states = ET.SubElement(frag_model, "limitStates")
    limit_states.text = "slight moderate extensive complete"
    
    for i in range(num_of_frag_func):
        frag_function = ET.SubElement(frag_model, "fragilityFunction")
        frag_function.attrib = {}
        frag_function.attrib["id"] = frag_func_id[i]
        frag_function.attrib["format"] = "continuous"
        frag_function.attrib["shape"] = "logncdf"

        imls = ET.SubElement(frag_function, "imls")
        imls.attrib = {}
        imls.attrib["imt"] = "PGA"
        imls.attrib["noDamageLimit"] = no_damage_lim[i]
        imls.attrib["minIML"] = min_IML[i]
        imls.attrib["maxIML"] = max_IML[i]

        slight = ET.SubElement(frag_function, "param")
        slight.attrib = {}
        slight.attrib["ls"] = "slight"
        slight.attrib["mean"] = slight_mean[i]
        slight.attrib["stddev"] = slight_stddev[i]

        moderate = ET.SubElement(frag_function, "param")
        moderate.attrib = {}
        moderate.attrib["ls"] = "moderate"
        moderate.attrib["mean"] = moderate_mean[i]
        moderate.attrib["stddev"] = moderate_stddev[i]

        extensive = ET.SubElement(frag_function, "param")
        extensive.attrib = {}
        extensive.attrib["ls"] = "extensive"
        extensive.attrib["mean"] = extensive_mean[i]
        extensive.attrib["stddev"] = extensive_stddev[i]
        
        complete = ET.SubElement(frag_function, "param")
        complete.attrib = {}
        complete.attrib["ls"] = "complete"
        complete.attrib["mean"] = complete_mean[i]
        complete.attrib["stddev"] = complete_stddev[i]
    
    print("\n========================================================")
    print("\nHere is your xml file for fragility model!\n")
    
    # Convert the XML to a pretty-printed string
    fragility_model = ET.tostring(root_frag_model, encoding='utf8', method='xml')
    print(BeautifulSoup(fragility_model, "xml").prettify())
    
    # Create the XML file
    tree_fragility_model = ET.ElementTree(root_frag_model)
    file = os.path.join(data_folder, "fragility_model" + ".xml")
    tree_fragility_model.write(file)
    
    print("\n================= Process Completed ====================")
    print("Saved " + file)

    return end_of_process()

# Create XML file for fragility model
def create_rupture_model_xml(eq_magnitude, eq_rake, hypo_lat, hypo_lon, hypo_depth, fault_dip, fault_upper_depth, fault_lower_depth, fault_coord):
    """ 
    Return xml file for earthquake rupture model
    
    """
    # Create the root elemet
    root_rupture_model = ET.Element('nrml')
    root_rupture_model.attrib = {}
    root_rupture_model.attrib["xmlns:gml"] = "http://www.opengis.net/gml"
    root_rupture_model.attrib["xmlns"] = "http://openquake.org/xmlns/nrml/0.4"
    
    # Create sub-elements and add to the root
    simple_fault = ET.SubElement(root_rupture_model, 'simpleFaultRupture')
    
    magnitude = ET.SubElement(simple_fault, "magnitude")
    magnitude.text = eq_magnitude

    rake = ET.SubElement(simple_fault, "rake")
    rake.text = eq_rake

    hypocenter = ET.SubElement(simple_fault, "hypocenter")
    hypocenter.attrib = {}
    hypocenter.attrib["lat"] = hypo_lat
    hypocenter.attrib["lon"] = hypo_lon
    hypocenter.attrib["depth"] = hypo_depth
    
    simple_fault_geom = ET.SubElement(simple_fault, "simpleFaultGeometry")            
    gml_linestring = ET.SubElement(simple_fault_geom, "gml:LineString")
    gml_poslist = ET.SubElement(gml_linestring, "gml:posList")
    gml_poslist.text = fault_coord

    dip = ET.SubElement(simple_fault_geom, "dip")  
    dip.text =fault_dip

    upper_depth = ET.SubElement(simple_fault_geom, "upperSeismoDepth")  
    upper_depth.text =fault_upper_depth

    lower_depth = ET.SubElement(simple_fault_geom, "lowerSeismoDepth")  
    lower_depth.text =fault_lower_depth
    
    print("\n========================================================")
    print("\nHere is your xml file!\n")    

    # Convert the XML to a pretty-printed string   
    rupture_model = ET.tostring(root_rupture_model, encoding='utf8', method='xml')
    print(BeautifulSoup(rupture_model, "xml").prettify())
    
    # Create the XML file
    tree_rupture_model = ET.ElementTree(root_rupture_model)
    file = os.path.join(data_folder, "rupture_model" + ".xml")
    tree_rupture_model.write(file)
    
    print("\n================= Process Completed ====================")
    print("Saved " + file)

    return end_of_process()

# Create job.ini file
def create_job_ini(job_desc, rupture_mesh_spacing, ref_vs30, depth_2pt5, depth_1pt0, gmpe, trunc_level, max_distance, num_gmf):
    """ 
    Return job.ini file for scenario damage calculation
    
    """
    file = os.path.join(data_folder, "job" + ".ini")
    output_path= os.path.join(os.getcwd(), output_folder)
    f = open(file,'w')
    f.write('[general]\n')
    f.write(f"description = {job_desc}\n")
    f.write('calculation_mode = scenario_damage\n')
    f.write('random_seed = 3\n')    
    
    f.write("\n[Rupture information]\n")
    f.write("rupture_model_file = rupture_model.xml\n")
    f.write(f"rupture_mesh_spacing = {rupture_mesh_spacing}\n")
    
    f.write("\n[Hazard sites]\n")
    
    f.write("\n[Exposure model]\n")
    f.write("exposure_file = exposure_model.xml\n")
    
    f.write("\n[Fragility model]\n")
    f.write("structural_fragility_file = fragility_model.xml\n")
    
    f.write('\n[Site conditions]\n')
    f.write('reference_vs30_type = measured\n')
    f.write(f'reference_vs30_value = {ref_vs30}\n')
    f.write(f'reference_depth_to_2pt5km_per_sec = {depth_2pt5}\n')
    f.write(f'reference_depth_to_1pt0km_per_sec = {depth_1pt0}\n')
    
    f.write('\n[Calculation parameters]\n')
    f.write(f'truncation_level = {trunc_level}\n')
    f.write(f'maximum_distance = {max_distance}\n')
    f.write(f'gsim = {gmpe}\n')
    f.write(f'number_of_ground_motion_fields = {num_gmf}\n')
    
    f.write('\n[output]\n')
    f.write(f'export_dir = {output_path}\n')
    f.close()
    print("\n========================================================")
    print("================= Process Completed ====================")
    print("Saved " + file)
    
    return end_of_process()

# Function to generate the XML file for the exposure model
def xml_exposure_model():
    user_inputs = ipt.exposure_model_user_input()

    return create_exposure_model_xml(*user_inputs)

# Function to generate the XML file for the fragility model
def xml_fragility_model():
    user_inputs = ipt.fragility_model_input()

    return create_fragility_model_xml(*user_inputs)

# Function to generate the XML file for the rupture model
def xml_rupture_model():
    user_inputs = ipt.rupture_model_input()

    return create_rupture_model_xml(*user_inputs)

# Function to generate the job.ini file
def job_ini():
    user_inputs = ipt.job_ini_input()

    return create_job_ini(*user_inputs)

# Function to clear the screen
def clear_screen():
    os.system("cls" if os.name=="nt" else "clear")

# Function to exit the program
def exit_program():
    return None

# Function to display the main menu and handle user selection
def main_menu():
    clear_screen()
    print("""Welcome to 'XML & job.ini File Generator'. Pick a file that you want to generate!
    0 - Main menu (this menu)
    1 - Exposure Model
    2 - Fragility Model
    3 - Earthquake Rupture Model
    4 - job.ini""")
    return program_pick()

# Function to handle user input and execute the selected program
def program_pick():
    file_map = {
        'c': exit_program,
        '0': main_menu,
        '1': xml_exposure_model,
        '2': xml_fragility_model,
        '3': xml_rupture_model,
        '4': job_ini
    }

    # Get user input for program selection
    user_pick = input("Pick 1 to 4 or 0 to go back to menu (or enter c to exit) : ")
    
    # Validate user input and execute the selected program
    while user_pick not in file_map:
        user_pick = input("Not a number from 0 to 4! (or enter c to exit) : ")
    return file_map[user_pick]()

def end_of_process():
    ipt.print_line_break()
    if ipt.yes_no_input("Do you want to generate other file? (Y/N) ") == "y":
        return main_menu()
    else:
        return exit_program()

In [5]:
# Run the program
# Displays the main menu, pick 1 - Exposure Model, and fill the parameters/settings
main_menu()

Welcome to 'XML & job.ini File Generator'. Pick a file that you want to generate!
    0 - Main menu (this menu)
    1 - Exposure Model
    2 - Fragility Model
    3 - Earthquake Rupture Model
    4 - job.ini
Pick 1 to 4 or 0 to go back to menu (or enter c to exit) : 1

Welcome to Exposure Model Generator!
Please fill in all of these parameters.
Exposure Model ID: exposure_model_1
Exposure Model Description: Perth Exposure Model
--------------------------------------------------------
STRUCTURAL COST
Do you have structural cost? (Y/N): N
--------------------------------------------------------
NONSTRUCTURAL COST
Do you have nonstructural Cost (Y/N): N
--------------------------------------------------------
BUSINESS COST
Do you have business Cost (Y/N): N
--------------------------------------------------------
CONTENTS COST
Do you have contents Cost (Y/N): N
--------------------------------------------------------
Do you have occupancy data? (Y/N): N
Do you have tag names? (Y/N): Y
Tag

In [6]:
# Displays the main menu, pick 2 - Fragility Model, and fill the parameters/settings
main_menu()

Welcome to 'XML & job.ini File Generator'. Pick a file that you want to generate!
    0 - Main menu (this menu)
    1 - Exposure Model
    2 - Fragility Model
    3 - Earthquake Rupture Model
    4 - job.ini
Pick 1 to 4 or 0 to go back to menu (or enter c to exit) : 2

Welcome to Fragility Model for Continuous Function Generator!
Please fill in all of these parameters.
Fragility Model ID: fragility_model_1
Loss type:
    1 - Structural
    2 - Non-structural
    3 - Business Interruption
    4 - Contents
    5 - Occupants
Pick 1 to 5 for your loss category: 1
Your loss category:  structural
Description of your fragility model: perth_fragility_model
How many fragility function do you have? 3

Fragility Function No  1

Fragility Function ID: wood
Damage Limit: 0.05
Minimum Intensity Measure Level: 0.0
Maximum Intensity Measure Level: 3.0
Slight damage - Mean: 0.2209
Slight damage - Standard deviation: 0.1572
Moderate damage - Mean: 0.3559
Moderate damage - Standard deviation: 0.2532
Exte

In [7]:
# Displays the main menu, pick 3 - Rupture Model, and fill the parameters/settings
main_menu()

Welcome to 'XML & job.ini File Generator'. Pick a file that you want to generate!
    0 - Main menu (this menu)
    1 - Exposure Model
    2 - Fragility Model
    3 - Earthquake Rupture Model
    4 - job.ini
Pick 1 to 4 or 0 to go back to menu (or enter c to exit) : 3

Welcome to Simple Rupture Model Generator!
Please fill in all of these parameters.
Earthquake magnitude: 6
Earthquake rake (-180 <= float <= 180): 90
Hypocenter latitude (-90 <= float <= 90): -31.93
Hypocenter longitude (-180 <= float <= 180): 116.06
Hypocenter depth (km) (float >= 0): 10
Fault dip (0 < float <= 90): 45
Fault upper depth (float >= 0): 3
Fault lower depth (float >= 0): 25
--------------------------------------------------------
Simple Fault Geometry
How many coordinates of the faults do you have? 2

Coordinate  1
latitude: -31.86
longitude: 116.063

Coordinate  2
latitude: -31.009
longitude: 116.058


Here is your xml file!

<?xml version="1.0" encoding="utf-8"?>
<nrml xmlns="http://openquake.org/xmlns/nr

In [8]:
# Displays the main menu, pick 4 - job.ini, and fill the parameters/settings
main_menu()

Welcome to 'XML & job.ini File Generator'. Pick a file that you want to generate!
    0 - Main menu (this menu)
    1 - Exposure Model
    2 - Fragility Model
    3 - Earthquake Rupture Model
    4 - job.ini
Pick 1 to 4 or 0 to go back to menu (or enter c to exit) : 4

Welcome to job.ini Generator!
Please fill in all of these parameters.
--------------------------------------------------------
General
Description of your calculation: Perth Scenario Damage
--------------------------------------------------------
Rupture Information
Rupture mesh spacing: 5
--------------------------------------------------------
Site conditions
Reference vs30 value (m/s): 820
Minimum depth (km) at which vs30 ≥ 2.5 km/s (z2.5): 5
Minimum depth (m) at which vs30 ≥ 1.0 km/s (z1.0): 100
--------------------------------------------------------
Calculation Parameters
Ground Motion Prediction Equation (GMPE): Allen2012
Level of trunction: 1
Maximum source-to-site distance (km): 100
Number of ground motion field

- All generated files (`exposure_model.xml`, `fragility_model.xml`, `rupture_model.xml`, and `job.ini`) will be stored in [data](./data/) folder.

---

## oq_calculation

- Input files for this program are generated from `job_xml_generator.py`.
- Make sure that all `xml` files defined in `job.ini` has the same name with `xml` file in the folder.
- To run this program, click `Run` button or push `shift+enter` on keyboard for each cell below.

In [ ]:
"""
The oq_calculation.py is a program aimed to performs OpenQuake calculations 
required for ground motion modeling and probability of damage distribution.

Inputs of this program are XML-based data and configuration file that is generated
by `job_xml_generator.py`. The calculation that run in this program is based on 
calculation mode defined in configuration file. This program will generate gmf_data.csv, 
avg_gmf.gmf, damages-rlzs.csv, aggrisk.csv, and risk_by_event.csv.
"""

In [9]:
# Import all modules and libraries
import os
from openquake.engine import engine
from openquake.calculators.export import hazard
from openquake.commonlib import logs
from openquake.calculators.base import calculators
from openquake.calculators.extract import extract
from openquake.calculators.export import risk

In [10]:
def main():
    # Run oq engine for the first time to make sure DB server is initialized
    os.system("oq engine")

    # Path is path for location where the "job.ini" is saved. 
    # job.ini should be saved in the same folder with all input file for the calculation
    path = os.path.join(os.getcwd(),"data", "job" + ".ini")

    # Calculation based on calculation mode defined in the job.ini
    with logs.init('job', path) as log:
        calc = calculators(log.get_oqparam(), log.calc_id)
        calc.run()  # run the calculator

    # Export output file and store it in directory defined in job.ini
    hazard.export_gmf_data_csv(("gmf_data","csv"),dstore=calc.datastore)
    hazard.export_avg_gmf_csv(("avg_gmf","csv"),dstore=calc.datastore)
    risk.export_damages_csv(("damages-rlzs","csv"),dstore=calc.datastore) 
    risk.export_aggrisk(("aggrisk","csv"),dstore=calc.datastore)
    risk.export_event_loss_table(("risk_by_event","csv"),dstore=calc.datastore)

In [11]:
# Run the program
main()

[2023-06-04 23:10:35 #166 INFO] Checksum of the inputs: 1758831342 (total size 9.29 MB)
[2023-06-04 23:10:35 #166 INFO] Running EventBasedCalculator with concurrent_tasks = 16
[2023-06-04 23:10:35 #166 WARNING] Entering SLOW MODE. You have 1.0 GB available, but the engine would like at least 0.5 GB per core, i.e. 4.0 GB: https://github.com/gem/oq-engine/blob/master/doc/faq.md
[2023-06-04 23:10:35 #166 WARNING] Found minIML=0 in D:\EMSC-4033-2023\ProjectPlans\ProjectPlan_u7582024\Project_Putri\data\fragility_model.xml, line 1, using 1e-10 instead
[2023-06-04 23:10:35 #166 WARNING] Found minIML=0 in D:\EMSC-4033-2023\ProjectPlans\ProjectPlan_u7582024\Project_Putri\data\fragility_model.xml, line 1, using 1e-10 instead
[2023-06-04 23:10:35 #166 WARNING] Found minIML=0 in D:\EMSC-4033-2023\ProjectPlans\ProjectPlan_u7582024\Project_Putri\data\fragility_model.xml, line 1, using 1e-10 instead
[2023-06-04 23:10:36 #166 INFO] Reading D:\EMSC-4033-2023\ProjectPlans\ProjectPlan_u7582024\Project_Pu

- There are 5 outputs generated by this program, including `gmf_data_X.csv`,`avg_gmf_X.csv`,`damages-rlzs-000_X.csv`,`aggrisk-_X.csv`, and `risk_by_event_X.csv`, with `X` is `calculation_id`. For example, if one of your output is `avg_gmf_160csv`, your `calculation_id` is `160`. These files are stored in [output](./output/) folder. 